# GenAI messages Python models

This file defines Python models for system instructions, input, and output messages.
These models are provided for reference only.

The model definitions are based on [Pydantic](https://github.com/pydantic/pydantic).

In [ ]:
pip install pydantic~=2.0

## Common Code

In [2]:
from enum import Enum
import json
from typing import Annotated, Any, List, Literal, Optional, Union
from pydantic import BaseModel, Field, RootModel

class TextPart(BaseModel):
    """
    Describes text content sent to or received from the model.
    """
    type: Literal['text']
    content: str = Field(description="Text content provided to or received from the model.")
    class Config:
        extra = "allow"

class ToolCallPart(BaseModel):
    """
    Describes a tool call requested by the model.
    """
    type: Literal["tool_call"] = Field(description="Content type.")
    id: str = Field(description="Unique identifier for the tool call.")
    name: str = Field(description="Name of the tool.")
    arguments: Optional[Any] = Field(description="Arguments for the tool call.")
    class Config:
        extra = "allow"

class ToolCallResponsePart(BaseModel):
    """
    Describes a result of tool call sent to the model.
    """
    type: Literal['tool_call_response'] = Field(description="Message type.")
    id: str = Field(description="Unique tool call identifier.")
    result: Optional[Any] = Field(description="Result of the tool call.")
    class Config:
        extra = "allow"

MessagePart = Annotated[
    Union[
        TextPart,
        ToolCallPart,
        ToolCallResponsePart,
        # Add other message part types here as needed,
        # e.g. image URL, image blob, audio URL, structured output, hosted tool call, etc.
    ],
    Field(discriminator='type'),
]

class ChatMessage(BaseModel):
    role: str
    parts: List[MessagePart]
    class Config:
        extra = "allow"


## `gen_ai.input.messages` model

Corresponding attribute: [`gen_ai.input.messages`](/docs/registry/attributes/gen-ai.md#gen-ai-input-messages).
JSON schema: [`gen_ai.input.messages.json`](../gen-ai-input-messages.json)

In [ ]:
class InputMessages(RootModel[List[ChatMessage]]):
    """

    Describes input messages sent to the model.
    """
    pass

# to print a JSON schema for the InputMessages model
print(json.dumps(InputMessages.model_json_schema(), indent=4))

## `gen_ai.output.messages` model

Corresponding attribute: [`gen_ai.output.messages`](/docs/registry/attributes/gen-ai.md#gen-ai-output-messages).
JSON schema: [`gen_ai.output.messages.json`](../gen-ai-output-messages.json)

In [ ]:
class FinishReason(str, Enum):
    """
    Describes the reason for finishing the generation.
    """

    STOP = "stop"
    LENGTH = "length"
    CONTENT_FILTER = "content_filter"
    TOOL_CALL = "tool_call"
    ERROR = "error"

class OutputMessage(ChatMessage):
    """
    Describes generated output.
    """
    finish_reason: Union[FinishReason, str] = Field(description="Reason for finishing.")

class OutputMessages(RootModel[List[OutputMessage]]):
    """
    Describes the output messages generated by the model.
    """
    pass

# to print a JSON schema for the OutputMessages model
print(json.dumps(OutputMessages.model_json_schema(), indent=4))